In [1]:
import numpy as np
import pandas as pd
import csv
import math
import random
import matplotlib.pyplot as plt
from world_cup_simulator import *

### Simulate group stage

In [ ]:
games_pd = pd.read_csv("matches.csv")
teams_pd = pd.read_csv("roster.csv")

In [ ]:
# Now, doing the Monte Carlo simulations
n = 1

for i in range(n):
    games = read_games("matches.csv")
    break
    teams = {}
    
    for row in [item for item in csv.DictReader(open("team_roster.csv"))]:
        teams[row['team']] = {'name': row['team'], 'rating': float(row['rating']), 'points': 0}
    
    simulate_group_stage(games, teams, ternary=False)
    
    for key in teams.keys():
        f = teams_pd['team'] == key
        teams_pd.loc[f, f"simulation{i+1}"] = teams[key]['points']
    
    teams_pd[f"simulation{i+1}"] = teams_pd.groupby('group')[f"simulation{i+1}"].rank(ascending=False)

In [ ]:
sim_cols = [i for i in teams_pd.columns if "simulation" in i]

In [ ]:
teams_pd['avg_pos'] = teams_pd[sim_cols].mean(axis=1)
teams_pd['median_pos'] = teams_pd[sim_cols].median(axis=1)
teams_pd['std_pos'] = teams_pd[sim_cols].std(axis=1)

In [ ]:
not_sim = [j for j in teams_pd.columns if "simulation" not in j]

In [ ]:
teams_pd[not_sim].sort_values(by=['group', 'avg_pos'])

### Simulating playoffs

In [2]:
playoff_games_pd = pd.read_csv("playoff_matches.csv")
playoff_teams_pd = pd.read_csv("playoff_roster.csv")

In [3]:
# Now, doing the Monte Carlo simulations
n = 10000
playoff_results_teams = []
playoff_results_stage = []

for i in range(n):
    overall_result_teams = dict()
    overall_result_stage = dict()
    games = read_games("playoff_matches.csv")
    teams = {}
    
    for row in [item for item in csv.DictReader(open("playoff_roster.csv"))]:
        teams[row['team']] = {'name': row['team'], 'rating': float(row['rating'])}
    
    simulate_playoffs(games, teams, ternary=True)
    
    playoff_pd = pd.DataFrame(games)
    
    # This is for collecting results of simulations per team
    for key in teams.keys():
        overall_result_teams[key] = collect_playoff_results(key, playoff_pd)
    playoff_results_teams.append(overall_result_teams)
    
    # Now, collecting results from stage-perspective
    overall_result_stage['Quarterfinals'] = playoff_pd.loc[playoff_pd['stage'] == 'eigths_finals', 'advances'].to_list()
    overall_result_stage['Semifinals'] = playoff_pd.loc[playoff_pd['stage'] == 'quarterfinals', 'advances'].to_list() + [None]*4
    overall_result_stage['Final'] = playoff_pd.loc[playoff_pd['stage'] == 'semifinals', 'advances'].to_list() + [None]*6
    overall_result_stage['third_place_match'] = playoff_pd.loc[playoff_pd['stage'] == 'semifinals', 'loses'].to_list() + [None]*6
    overall_result_stage['fourth_place'] = playoff_pd.loc[playoff_pd['stage'] == 'third_place', 'loses'].to_list() + [None]*7
    overall_result_stage['third_place'] = playoff_pd.loc[playoff_pd['stage'] == 'third_place', 'advances'].to_list() + [None]*7
    overall_result_stage['second_place'] = playoff_pd.loc[playoff_pd['stage'] == 'final', 'loses'].to_list() + [None]*7
    overall_result_stage['Champion'] = playoff_pd.loc[playoff_pd['stage'] == 'final', 'advances'].to_list() + [None]*7
    
    playoff_results_stage.append(overall_result_stage)

In [4]:
results_teams = pd.DataFrame(playoff_results_teams)

In [5]:
results_teams['Brazil'].value_counts(normalize=True)

Quarterfinals    0.3814
Round_of_16      0.1754
Champion         0.1746
Third_place      0.1628
Final            0.0573
Fourth_place     0.0485
Name: Brazil, dtype: float64

In [6]:
results_stage = pd.DataFrame(playoff_results_stage)

In [10]:
results_stage['Quarterfinals'].value_counts(normalize=True)

[Netherlands, Argentina, Spain, Brazil, England, Denmark, Belgium, Portugal]           0.0548
[Netherlands, Argentina, Spain, Brazil, England, Denmark, Germany, Portugal]           0.0439
[Netherlands, Argentina, Spain, Brazil, Ecuador, Denmark, Belgium, Portugal]           0.0330
[Netherlands, Argentina, Spain, Brazil, England, Denmark, Belgium, Switzerland]        0.0299
[Netherlands, Argentina, Croatia, Brazil, England, Denmark, Belgium, Portugal]         0.0290
                                                                                        ...  
[United States, France, Croatia, Uruguay, England, Poland, Germany, Switzerland]       0.0001
[United States, Argentina, Croatia, Uruguay, Ecuador, Poland, Belgium, Switzerland]    0.0001
[United States, France, Spain, Uruguay, England, Poland, Germany, Portugal]            0.0001
[United States, Argentina, Croatia, Uruguay, Ecuador, Poland, Belgium, Portugal]       0.0001
[United States, France, Croatia, Brazil, Ecuador, Poland, Be